### Packages ###

In [1]:
#load "../Data/Data.fsx"

#r "nuget: FSharp.Stats"
#r "nuget: Plotly.NET"

open Data
open FSharp.Stats
open FSharp.Stats.Algebra.LinearAlgebra


The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages FSharp.Stats, 0.4.11 Plotly.NET, 4.0.0

### Types ###

In [2]:
type Vec = Vector<float>

type Mat = Matrix<float>

### Helper functions ###

In [3]:
let estimateTheta (X: Mat) (Y: Vec) =
    let XT = X |> Matrix.transpose

    ((XT * X) |> Inverse) * XT * Y

let train (f: Featurizer) (data: Obs seq) =
    let Yt, Xt =
        data |> Seq.toList |> List.map (fun obs -> float obs.Cnt, f obs) |> List.unzip

    let thetas = estimateTheta (matrix Xt) (vector Yt)

    thetas, (fun obs -> Vector.dot (f obs |> vector) thetas)



### Test featurizers ###

In [4]:

let f1 (obs: Obs) = [ 1.0; obs.Instant |> float ]

let f2 (obs: Obs) = [ 1.0; obs.Instant; float obs.Temp ]

let f3 (obs: Obs) =
    [ 1.0; obs.Instant; float obs.Temp; float obs.Windspeed ]


let f4 (obs: Obs) =
    [ 1.0; obs.Instant; float obs.Temp; float obs.Windspeed; float obs.Hum ]

### Let's load some data ###

In [5]:
let allData = loadData()

let training, validation = allData |> splitData 0.7

### Let's train some models ###

In [6]:
let thetas1, model1 = train f1 training
let thetas2, model2 = train f2 training
let thetas3, model3 = train f3 training
let thetas4, model4 = train f4 training

### Let's evaluate them ###

In [7]:
evaluate model1 validation |> printfn "Model1: %A \n"
evaluate model2 validation |> printfn "Model2: %A \n"
evaluate model3 validation |> printfn "Model3: %A \n"
evaluate model4 validation |> printfn "Model4: %A \n"


Model1: 1201.846758 

Model2: 809.3199494 

Model3: 792.5169779 

Model4: 743.3872076 

